In [1]:
using DataFrames
using CSV

In [2]:
withdraw_transactions_df = CSV.read("../data/tornado_withdraw_df.csv", DataFrame)
deposit_transactions_df = CSV.read("../data/deposit_transactions.csv", DataFrame);

In [3]:
ENV["COLUMNS"]=10000

10000

In [4]:
function same_gas_price(withdrawal_address_transaction, deposit_dataframe)
    same_gas_price = []
    for row in eachrow(deposit_dataframe)
        if withdrawal_address_transaction.gas_price == row.gas_price && withdrawal_address_transaction.block_timestamp > row.block_timestamp
            push!(same_gas_price, row.hash)
        end
    end
    same_gas_price
end

same_gas_price (generic function with 1 method)

In [5]:
same_gas_price(withdraw_transactions_df[1,:], deposit_transactions_df)

41-element Array{Any,1}:
 "0x654052299f351da7dca61a3af77bc2e1e30f9fa8c1304ca1f995771709c85317"
 "0x65e5c538d2b7573b6572283ffd31e5943a047b54c4a402a493897c008b6c4431"
 "0x555fc2c856f95c7b4529888faf31152ab64ef31a6a0ce3dac3a1a5e8674f05e7"
 "0x7e8a926e2accf193cc02c7946d34dd2a481de02e497bd8a1d993c7160544f26e"
 "0x588a2c8cb180bafb03e911ed657230b756a906a84594354c8839a77151ec5f50"
 "0x444ff60f12e018f77adb8f72da159f9572b59f98bc9816a45b259ffd6155af84"
 "0x9bdf0bac379eb49ec9b9d2dc81366daacdd93ecf1dbb7952e1740a9212ce2d60"
 "0x700cbbd11c3deda9c575b1955b7cd4d718acc561b0fe30c37c84c263361acc2a"
 "0x5a809cacc658bb8b8f7d3a12307772348b254c16b6fe29a91489e540d898d3f5"
 "0x568a97fb3130c2220af1cf975560d263a798c31bb8d564f5309226a46e369ff9"
 "0x8d802c2869e2c42d885d1ce9ae1891cfbff340e772d77688b2881ac8b3c6299d"
 "0xa0d948b801596792f125e8dbfa083933db6f0ec956e5f9aa4f5153f0e0668c21"
 "0x872383603f292d6eefdeac3395bc6f91822e29cbbc06e14e9b9b0b9b93984aae"
 ⋮
 "0xa12f4a2b173a32d7a719a6d28801b889e5669508cde66e0cca0ccdb0dd

In [6]:
function apply_same_gas_price(deposit_dataframe, withdraw_dataframe)
    withdrawal_to_deposit = Dict()
    for withdraw_row in eachrow(withdraw_dataframe)
        withdrawal_to_deposit[withdraw_row.hash] = same_gas_price(withdraw_row, deposit_dataframe)
    end
    withdrawal_to_deposit
end     

apply_same_gas_price (generic function with 1 method)

In [ ]:
link_same_gas = apply_same_gas_price(deposit_transactions_df, withdraw_transactions_df);

In [36]:
hashes = collect(keys(link_same_gas));

In [37]:
num_deposit_links = [length(link_same_gas[key]) for key in collect(keys(link_same_gas))];

In [46]:
link_gas_price_df = DataFrame("withdraw_hash"=>hashes, "linked_deposits"=>num_deposit_links)

,withdraw_hash,linked_deposits
,Any,Int64
1,0xe5bb17e135adbd5c381f626920779db580f4dbb4b2c8ee557df7fc17065f80d7,79
2,0x86cbefe25c942108732dd4aded45fa322a777f3a31c6b660343934b2cbb971d6,69
3,0xe02d94bb77ec0ebf35b99a3040f7d6da3dcfe3c97c728dfd9ac4d08fdc3a1394,7
4,0x5f863ded4ac679e4075ef59097067e96182e20ce0479390d0a5613fd10443498,16
5,0xbbfc2bec5bb8079f8b9aff36bc18d55d0ed22fd01dccf1b28c16b8cf27351bd2,26
6,0x6ed0ed9f7bb86055430d669a01209723762994d859e4aeb9ea4cf1b152d6ebdc,11
7,0x9011dff9eac19b42a50c0b84a377d724b94b7959748853c9e228bf90d166c414,77
8,0x004982c4e614d167780d97e29329437b9f185e37bdc3c2e62179be11d7ab4f10,52
9,0xe150246c892c55cb330fb06c77f3bd016f051958ccec4d8452363012614c4273,106
